In [1]:
from bs4 import BeautifulSoup
import requests
import csv

In [2]:
def generateUrl(query):
    query = query.replace(' ', '+')
    # Generate Amazon URL from query
    url1 = 'https://www.amazon.co.uk/s?k=' + query
    
    # Generate TechBuyer URL from query
    url2 = 'https://www.techbuyer.com/uk/catalogsearch/result/?q=' + query
    
    # Generate scientific laboratory supplies URL from query
    url3 = 'https://www.scientificlabs.co.uk/search/' + query
    
#     # Generate RS URL from query
#     url4 = 'https://uk.rs-online.com/web/c/?searchTerm=' + query
    
    # Generate Farnell URL from query
    url4 = 'https://uk.farnell.com/search?st=' + query
    return url1, url2, url3, url4

def getData(url):
    HEADERS = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'}
    webpage = requests.get(url, headers=HEADERS)
    soup = BeautifulSoup(webpage.content, "lxml")
    return soup

def getAmazonProducts(bs):
    # Scrape product names
    names = bs.findAll('span',{'class':'a-size-medium a-color-base a-text-normal'})
    nameList = []
    for name in names:
        nameList.append(name.get_text())
    # Scrape product prices
    prices = bs.findAll('span',{'class':'a-offscreen'})
    priceList = []
    for price in prices:
        priceList.append(price.get_text())
    # Scrape product urls
    urls = bs.findAll('a',{'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})
    urlList = []
    for url in urls:
        url = 'https://www.amazon.co.uk/' + url.attrs['href']
        urlList.append(url)
    # Store scraped data in a dictionary
    n = len(nameList)
    products = {}
    for i in range(n):
        details = {}
        details['price'] = priceList[i]
        details['url'] = urlList[i]
        products[nameList[i]] = details
    result['Amazon'] =  products
    return result

def getTechbuyerProducts(bs):
    # Scrape product names
    names = bs.findAll('a',{'class':'products__item-link'})
    nameList = []
    for name in names:
        nameList.append(name.get_text())
    # Delete the "\n"
    temp = []
    for n in nameList:
        temp.append(n.strip())
    nameList = temp

    # Scrape product prices
    prices = bs.findAll('span',{'class':'price'})
    priceList = []
    for price in prices:
        priceList.append(price.get_text())

    # Scrape product urls
    urls = bs.findAll('a',{'class':'products__item-link'})
    urlList = []
    for url in urls:
        urlList.append(url.attrs['href'])
    # Store scraped data in a dictionary
    n = len(nameList)
    products = {}
    # Store scraped data in a dictionary
    n = len(nameList)
    products = {}
    for i in range(n):
        details = {}
        details['price'] = priceList[i]
        details['url'] = urlList[i]
        products[nameList[i]] = details
    result['Techbuyer'] =  products
    return result

def getScientificLabsProduct(bs):
    nameList = []
    urlList = []
    sizeList = []
    priceList = []

    # Scrape product details
    products = bs.findAll('div', {'class':'table-row'})
    for product in products:
        name_a = product.find('a',href=True, title=True, style="color:black;", target=True)
        href = name_a.get('href')
        name = name_a.get('title')
        size_div = product.find('div',{'class':'unit-size mobileGridCloserRows flex-100 indentedGridRow mobileDisplayContents range-unit'})
        size = size_div.find('span', id=True).get_text()
        price = product.find('span',{'class':'gridDisplayBlock'}).get_text()

        nameList.append(name)
        urlList.append(href)
        sizeList.append(size)
        priceList.append(price)

    n = len(products)
    product = {}
    for i in range(n):
        details = {}
        details['price'] = priceList[i]
        details['size'] = sizeList[i]
        details['url'] = urlList[i]

        product[nameList[i]] = details
    result['ScientificLaboratorySupplies '] =  product
    return result

# Test Result

In [3]:
query = 'Acetone'
url1, url2, url3, url4 = generateUrl(query)
result = {}
getAmazonProducts(getData(url1))
getTechbuyerProducts(getData(url2))
getScientificLabsProduct(getData(url3))

{'Amazon': {'Saviland Gel Nail Polish Remover Kit - 250ML Nail Polish Remover with Nail Clips, Nail Tools,Cuticle Pusher, 500PCS Cotton Pad Nail Remover Manicure Kit for Salon Home DIY': {'price': '£9.99',
   'url': 'https://www.amazon.co.uk//sspa/click?ie=UTF8&spc=MTo4MTA0NzY3NzA3NjI5NzI1OjE2Njk2NzM5Mzk6c3BfYXRmOjIwMDY1MjE1MTQ2MjA0OjowOjo&url=%2FAcetone-Superior-Quality-Polish-Remover%2Fdp%2FB0182QCU7E%2Fref%3Dsr_1_1_sspa%3Fkeywords%3DAcetone%26qid%3D1669673939%26sr%3D8-1-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGY%26psc%3D1%26smid%3DAYGGMWI8Z0G1N'},
  'Magic Nail Polish Remover, Professional Easily & Quickly Removes for Natural, Gel Nails without Hurting Nails and No Need For Foil, Soaking Or Wrapping': {'price': '£13.50',
   'url': 'https://www.amazon.co.uk//sspa/click?ie=UTF8&spc=MTo4MTA0NzY3NzA3NjI5NzI1OjE2Njk2NzM5Mzk6c3BfYXRmOjIwMDc3NTgzNzU2NTA0OjowOjo&url=%2FMylee-Acetone-Polish-Remover-Quality%2Fdp%2FB0182QGCJQ%2Fref%3Dsr_1_2_sspa%3Fkeywords%3DAcetone%26qid%3D1669673939%26sr%3D8-2

# Test For Farnell

In [4]:
bs = getData(url4)

nameL = []
priceL = []
urlL = []

# Scrape product details
descriptions = bs.findAll('td', {'class':'description enhanceDescClmn'})
descriptions[0]

<td class="description enhanceDescClmn">
<input class="hVal" type="hidden" value="MULTICOMP PRO"/>
<a href="https://uk.farnell.com/multicomp-pro/mc002236/cleaner-bottle-1l/dp/2854870?st=Acetone" title="Cleaner, Acetone Solvent, For 3D Printing, Bottle, 1L Volume">
<p class="productDecription">Cleaner, Acetone Solvent, For 3D Printing, Bottle, 1L Volume</p>
</a>
<p class="manufacturerName">MULTICOMP PRO</p>
<div class="plpBestSellerWraper" id="vn_bestSeller_182663039">
</div>
<div class="displayNone" data-preflagordercode-2854870="true" id="prevPF">
<span class="icon-info-circled ppf-infoIcon"></span>
<span class="ppf-txt">You previously purchased this product.
			<a href="https://uk.farnell.com/webapp/wcs/stores/servlet/PFOrderSearch?catalogId=15001&amp;searchField=orderCode&amp;storeId=10151&amp;langId=44&amp;searchValue=2854870" onclick="pf.tagging.VOHflgClick('plp', 2854870)">View in Order History</a>
</span>
</div>
</td>

In [5]:
for d in descriptions:
    name = d.find('a', href=True).get('title')
    url = d.find('a', href=True).get('href')
    nameL.append(name)
    urlL.append(url)
priceCol = bs.findAll('td',{'class':'listPrice enhanceQtyColumn'})
print(priceCol[0])

<td class="listPrice enhanceQtyColumn">
<div data-ref="182663039" id="vn_price_182663039">
<p class="price showPartList showFirst5" data-maxprices="5" id="priceList_0">
<span class="priceBreak data-product-pricerow-products-0">
<span class="qty" data-price="products-0-price-priceList-{priceNumber}-minQtyDesc">1+</span>
<span class="vatSwitchBox" data-price="products-0-price-listPrice">
<span class="vatExcl" data-price="products-0-price-priceList-{priceNumber}-priceFormatted">£11.52</span>
<span class="vatIncl" data-price="products-0-price-priceList-{priceNumber}-vatPriceFormatted">£13.82</span>
</span>
</span>
</p>
<input id="2854870" style="display:none;" type="textarea" value="2854870**false**false**true**true**false**true**[[1,11.52]]"/>
<script type="text/javascript">
	if(!priceBreakArrays)
	{
		var priceBreakArrays =[];
	}
		priceBreakArrays['182663039']=[[1,11.52]];
</script>
</div>
<div class="priceWidthHold"></div>
</td>


In [6]:
for i in priceCol:
    price = i.find('span', {'class':'vatExcl'}).get_text()
    priceL.append(price)

In [8]:
print(len(priceL), len(nameL), len(urlL))

9 9 9


In [12]:
n = len(nameL)
product = {}
for i in range(n):
    details = {}
    details['price'] = priceL[i]
    details['url'] = urlL[i]

    product[nameL[i]] = details
# result['Farnell '] =  product
product

{'Cleaner, Acetone Solvent, For 3D Printing, Bottle, 1L Volume': {'price': '£11.52',
  'url': 'https://uk.farnell.com/multicomp-pro/mc002236/cleaner-bottle-1l/dp/2854870?st=Acetone'},
 'Cleaner, Acetone Solvent, Fast Drying and Zero Residue, 3D Printing, Can, 945 ml': {'price': '£10.44',
  'url': 'https://uk.farnell.com/mg-chemicals/434-1l/acetone-solvent-can-945ml/dp/2811935?st=Acetone'},
 'Dispensing Bottle, Dissipative, ESD, Pump, Acetone Printed, Yellow, 180ml DurAstatic Series': {'price': '£18.98',
  'url': 'https://uk.farnell.com/menda/35277/dispenser-pump-bottle-yellow-180ml/dp/2907825?st=Acetone'},
 'Dispensing Bottle, Dissipative, ESD, Pump, Acetone Printed, Blue, 240ml DurAstatic Series': {'price': '£29.39',
  'url': 'https://uk.farnell.com/menda/35288/dispenser-pump-bottle-blue-240ml/dp/2907833?st=Acetone'},
 'Cleaner, Surface, Acetone/Butanone, PVC-U, Tangit, Can, 1 l': {'price': '£13.72',
  'url': 'https://uk.farnell.com/tangit/44267/cleaning-pvc-u-tangit-detergente/dp/150